# Demo de análise preditiva com Machine Learning

## Importação das bibliotecas necessárias

Esse é o `requirements.txt` utilizado no projeto:

    asttokens==2.4.1
    attrs==23.2.0
    comm==0.2.2
    contourpy==1.2.0
    cycler==0.12.1
    debugpy==1.8.1
    decorator==5.1.1
    exceptiongroup==1.2.0
    executing==2.0.1
    fastjsonschema==2.19.1
    fonttools==4.49.0
    ipykernel==6.29.3
    ipython==8.22.2
    jedi==0.19.1
    joblib==1.3.2
    jsonschema==4.21.1
    jsonschema-specifications==2023.12.1
    jupyter_client==8.6.1
    jupyter_core==5.7.2
    kiwisolver==1.4.5
    matplotlib==3.8.3
    matplotlib-inline==0.1.6
    nbformat==5.10.2
    nest-asyncio==1.6.0
    numpy==1.26.4
    packaging==24.0
    pandas==2.2.1
    parso==0.8.3
    pexpect==4.9.0
    pillow==10.2.0
    platformdirs==4.2.0
    plotly==5.19.0
    prompt-toolkit==3.0.43
    psutil==5.9.8
    ptyprocess==0.7.0
    pure-eval==0.2.2
    Pygments==2.17.2
    pyparsing==3.1.2
    python-dateutil==2.9.0.post0
    pytz==2024.1
    pyzmq==25.1.2
    referencing==0.33.0
    rpds-py==0.18.0
    scikit-learn==1.4.1.post1
    scipy==1.12.0
    six==1.16.0
    stack-data==0.6.3
    tenacity==8.2.3
    threadpoolctl==3.3.0
    tornado==6.4
    traitlets==5.14.2
    tzdata==2024.1
    wcwidth==0.2.13

In [ ]:
import pandas
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from scipy.stats import randint

## Importação do dataset

Aqui criamos um dataframe Pandas com o conteúdo do arquivo csv. O dataframe é a estrutura padrão de manejo de dados para bibliotecas de ML em Python

In [ ]:
dataset = pandas.read_csv("churn.csv")

## Exploração dos dados

Uma vez que o dataset tenha sido carregado, podemos começar uma análise exploratória, para entendermos a estrutura e os tipos dos nossos dados, e tentar auferir possíveis co-relações. Essa exploração também pode ser feita antes da importação dos dados, utilizando ferramentas de BI como planilhas, PowerBI e etc. Mas também temos algumas funções interessantes para essa exploração dentro do próprio Python.

O primeiro passo é verificar a qualidade dos dados, certificando que tenhamos o mínimo possível de células sem informação. Para isso, podemos utilizar a função `info()` do dataset, que nos dá uma visão resumida dos dados, incluindo os tipos de colunas e a quantidade de dados nulos, caso haja:

In [ ]:
dataset.info()


A função `head()` mostra o cabeçalho e os 5 primeiros registro do dataset e é muito útil para entendermos a estrutura dos dados. É um ótimo ponto de partida para visualizarmos claramente quais informações estão disponíveis e o que estamos tentando categorizar:

In [ ]:
dataset.head()

### Plotando gráficos

Outra ferramenta interessante é a possibilidade de plotar gráficos que representem nossos dados, para fácil vizualização.

Um dos tipos mais úteis de gráfico é o histograma, que é ótimo para propriedades qualitativas. Podemos, por exemplo, criar e exibir um histograma co-relacionando o país de nosso clientes e a taxa de churn assim:

In [ ]:
px.histogram(dataset, x = "churn", text_auto = True)

In [ ]:
px.histogram(dataset, x = "pais", text_auto = True, color = "churn", barmode = "group")

Outro tipo de gráfico bem últil é o BoxPlot, ideal para valores numéricos, pois mostra muitas informações de uma só vez, inclusive outliers. Podemos plotar um gráfico Box com os salários dos clientes assim:

In [ ]:
px.box(dataset, x="salario_estimado", color="churn")

## Pré-processamento

Com os dados compreendidos, podemos iniciar o processo de limpeza e transformação desses dados.

### Limpeza e separação
Vamos começar apagando a coluna *id_cliente* que não vai ser utilizada.

In [ ]:
dataset = dataset.drop("id_cliente", axis=1)

Então separamos o dataset em dois conjuntos:
- **x**: dataset contendo todas as colunas de dados que serão utilizadas para treinar o modelo
- **y**: dataset contendo apenas a variável **churn**, que é o que queremos predizer

In [ ]:
y = dataset["churn"]
x = dataset.drop("churn", axis=1)

Aqui nós salvamos os nomes das colunas do dataset para uso futuro, pois quando o dataset for transformado para o processo de treinamento do modelo, essa informação será perdida.

In [ ]:
nomes_colunas = x.columns

### Transformação

Uma vez que os datasets tenham sido limpos e separados, precisamos tranformar os dados categóricos em numéricos, para que o modelo possa classificá-los.

Como pretendemos utilizar o modelo de **Árvore de Decisão**, vamos utilizar um codificador chamado **OneHotEncoder**. O que ele faz é criar novas colunas para cada categoria textual no dataframe, marcar a coluna correspondente com o valor 1 e as colunas restantes com 0. Também vamos configurá-lo para, caso a coluna só tenha dois valores, como `sexo_biologico`, ele mantém uma coluna para apenas um dos valores e marca o registro com 1 caso positivo e 0 caso negativo:

In [ ]:
encoder = OneHotEncoder(drop="if_binary")

O codificador criado no passo anterior será agora utilizado por uma **função de transformação**. Essa é a função que efetivamente aplicará as mudanças no dataset, de acordo com as configurações do codificado criado anteriormente. Vamos criar uma instância dessa função, e como parâmetros vamos passar:
- Uma tupla contendo:
    - O codificador
    - A lista de nomes de colunas a serem transformadas
- Uma instrução do que fazer com os demais campos (nesse caso, nada)
- Quanto de limite de sparsing aplicar. Devemos manter em 0 para modelos de Árvore de Decisão

In [ ]:
transformer = make_column_transformer(
    (
        encoder,
        [
            "pais",
            "sexo_biologico"
        ]
    ),
    remainder="passthrough",
    sparse_threshold=0
)

Agora que temos nosso transformer iniciado e configurado com o nosso codificador, podemos aplicá-lo ao dataset de treino:

In [ ]:
x = transformer.fit_transform(x)

O objeto retornado pela função de transformação é um Array NumPy. Vamos transformá-lo de volta em um Dataframe Pandas. Vamos também utilizar a função `get_feature_names_out()` do transformador com a lista de nomes de colunas que salvamos no começo da demo para dar nomes úteis às colunas do Dataframe:

In [ ]:
x = pandas.DataFrame(x, columns=transformer.get_feature_names_out(nomes_colunas))

E vamos checar o resultado:

In [ ]:
x.head()

### Transformando o dataset alvo

No nosso caso específico, a coluna *churn* já está em valor numérico. Mas caso ela não estivese, teríamos que passar o nosso dataset objetivo (y) por uma função transformadora também. Isso pode ser feito com a classe `LabelEncoder`. Essa classe só deve ser utilizada para transformar o dataset alvo, já que o resultado é um array de categorias. Caso você passe o dataset alvo pelo LabelEncoder, lembre-se de que você não precisa transformar ele num Dataframe Pandas. O modelo pode utilizar o dataset de alvo como um array.

In [ ]:
y = LabelEncoder().fit_transform(y)

### Dividindo o dataset em dados para treino e dados para teste

Depois de explorar e transformar os dados conforme nossa necessidade, só nos resta dividir nosso dataset em dois datasets separados. Um deles, o maior, será utilizado para treinar o modelo. O segundo, um pouco menor, será utilizado para testar o modelo. É importante que o modelo nunca tenha *visto* os dados de teste, para podermos ter certeza de que ele não apenas decorou os dados.

A biblioteca de modelos do pacote SciKit tem uma função própria para fazer essa divisão: `train_test_split()`. Vamos executá-la passando como parâmetros os dois datasets (x de treino e y de objetivo) e qual é o dataset objetivo com a propriedade `stratify`:

In [ ]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, stratify=y)

## Criando e testando modelos

Com os datasets transformados e separados podemos finalmente começar a trabalhar nos modelos de predição.

### Modelo Dummy

Vamos começar setando um parâmetro mínimo de qualidade para os nossos modelos. Esse mínimo vai ser setado por um modelo chamado **Dummy**. O que esse modelo faz é analisar os valores que tentamos predizer e simplesmente escolher sempre o valor mais prevalente, assim *"acertando"* na maioria dos casos.

Para utilizar o model, instânciamos a classe `DummyClassifier` e então *inserimos* os datasets transformados de dados e respostas:

In [ ]:
dummy = DummyClassifier()
dummy.fit(x_treino, y_treino)

Em nosso Dataset de 10 mil registros, 7963 registros tem um valor positivo para churn. O modelo deve identificar que a maioria dos valores é positivo e então usar positivo em todos os testes. Então, se aplicarmos o modelo ao Dataset, deveríamos ter uma taxa de acerto de aproximadamente **79.63%**:

In [ ]:
dummy.score(x_teste, y_teste)

### Modelo de Árvore de Decisão

Com o baseline de aproximadamente 79% definido, já temos como avaliar a qualidade de um modelo real. Vamos importar o modelo `DecisionTreeClassifier` e aplicá-lo aos nossos dados, de maneira similar com o que fizemos com o modelo Dummy:

In [ ]:
arvore = DecisionTreeClassifier()
arvore.fit(x_treino, y_treino)

E vamos avaliar a qualidade das previsões, lembrando que o modelo Dummy acertou 79%:

In [ ]:
arvore.score(x_teste, y_teste)

#### Ajuste do modelo

Apesar do resultado ser melhor do que do modelo Dummy, não foi por uma margem expressiva. Para melhorar isso, podemos ajustar diversos parâmetros do modelo. Mas podemos começar a análise desenhando a árvore de decisão resultante:

In [ ]:
plt.figure(figsize=(100,10))
plot_tree(arvore, filled=True, fontsize=7, class_names=["nao", "sim"], feature_names=x_treino.columns)

Podemos perceber que a árvore ficou bem grande e confusa. Vamos limitar o tamanho da árvore e checar se o resultado melhora. Para isso utilizamos um parâmetro chamado `max_depth`, que define o máximo de escolhas que a árvore pode fazer antes de *tomar uma decisão*. Vamos re-instanciar a árvore, mas dessa vez setando o `max_depth` em 15, re-aplicar ao dataset...

In [ ]:
arvore = DecisionTreeClassifier(max_depth=15)
arvore.fit(x_treino, y_treino)

... e verificar o score novamente:

In [ ]:
arvore.score(x_teste, y_teste)

Podemos perceber que houve uma melhora. Nesse caso, poderíamos testar diferentes valores até encontrar o que retorna o melhor resultado, o que pode ser bem trabalhoso, ou utilizar a classe `RandomizedSearchCV` para testar possíveis valores automaticamente.

Para utilizar a classe `RandomizedSearchCV` precisamos antes de um dicionário com os parâmetros para os quais vamos buscar o melhor resultado e o range de valores a tentar:

In [ ]:
param_distribution = {
    "max_depth": randint(1, 15),
    "criterion": ["gini", "entropy", "log_loss"],
    "splitter": ["best", "random"]
}

Com o dicionário pronto, vamos instanciar a classe passando o tipo de modelo para o qual queremos buscar os melhores parâmetros (**DecisionTreeCalssifier** nesse caso), quantas iterações o buscador deve fazer e, finalmente, o dicionário com os parâmetros a serem testados:

In [ ]:
search = RandomizedSearchCV(
    estimator=DecisionTreeClassifier(),
    n_iter=50,
    param_distributions=param_distribution
)

Com a instância gerada, vamos aplicar os dados de treino de deixar ela fazer sua mágica:

In [ ]:
search.fit(x_treino, y_treino)

O resultado é um objeto do tipo **estimator**, que tem entre seus parâmetros o campo `best_param_`, um dicionário com a informação que buscamos:

In [ ]:
search.best_params_

Agora podemos re-configurar nossa árvore com o melhor `max_depth` possível...

In [ ]:
arvore = DecisionTreeClassifier(max_depth=5, criterion="gini", splitter="best")
arvore.fit(x_treino, y_treino)

... e testar novamente:

In [ ]:
arvore.score(x_teste, y_teste)

Vamos ver como ficou nossa árvore definitiva:

In [ ]:
plt.figure(figsize=(100,10))
plot_tree(arvore, filled=True, fontsize=7, class_names=["nao", "sim"], feature_names=x_treino.columns)

Com o modelo criado, treinado e otimizado, podemos finalmente utilizá-lo para predizer um dado. Vamos começar criando um dicionário que represente uma possível nova linha no nosso dataset e então transformá-lo em um dataframe:

In [ ]:
novo_dado = {
    "score_credito": [619],
    "pais": ["França"],
    "sexo_biologico": ["Mulher"],
    "idade": [42],
    "anos_de_cliente": [2],
    "saldo": [0.00],
    "servicos_adquiridos": [1],
    "tem_cartao_credito": [1],
    "membro_ativo": [1],
    "salario_estimado": [101348.88]
}

novo_dado = pandas.DataFrame(novo_dado)

Depois disso, precisamos aplicar a mesma função de transformação utilizada para treinar o modelo e converter o resultado novamente em um dataframe

In [ ]:
novo_dado = transformer.transform(novo_dado)
novo_dado = pandas.DataFrame(novo_dado, columns=transformer.get_feature_names_out(nomes_colunas))

E estamos finalmente prontos para pedir que nosso modelo tente predizer um dado

In [ ]:
arvore.predict(novo_dado)